In [1]:
import sys
sys.path.insert(0, "..")
import snscrape.modules.twitter as sntwitter
from fastapi.encoders import jsonable_encoder
import json
import pandas as pd
import numpy as np
from pymongo import MongoClient
import datetime
import re
from config import conf

df = pd.read_excel('../datasets/deputes.xlsx', sheet_name="deputes-active")
df_party = pd.read_excel('../datasets/deputes.xlsx', sheet_name="parties-politiques")

In [8]:
df_party

,group,twitter
0,AGIR-E,AgirEnsemble_AN
1,DEM,MoDem
2,DLF,DLF_Officiel
3,EDS,edsassnat
4,FI,FranceInsoumise
5,GDR,deputesPCF
6,GE,GEcologie
7,LAREM,enmarchefr
8,LDS,Liguedusud
9,LND,NDemocrates_


In [2]:
df1 = df.copy()

def get_sex(civil):
    if civil.upper() == 'M.':
        return 'm'
    else:
        return 'f'

def fix_account_name(input):
    if type(input) == str:
        return input.replace('@', '')
    else:
        return input
  

df1['sex'] = df1.civ.apply(get_sex)
df1['twitter'] = df1.twitter.apply(fix_account_name)
df1

,id,legislature,civ,nom,prenom,naissance,age,groupe,groupeSansNI,groupeAbrev,...,facebook,website,nombreMandats,experienceDepute,scoreParticipation,scoreParticipationSpecialite,scoreLoyaute,scoreMajorite,dateMaj,sex
0,PA267355,15.0,M.,Herth,Antoine,1963-02-14,58.0,Agir ensemble,Agir ensemble,AGIR-E,...,antoine.herth,www.antoine-herth.fr/,4.0,19 ans,0.14,0.21,0.937,0.767,2021-08-22,m
1,PA267780,15.0,Mme,Firmin Le Bodo,Agnès,1968-11-20,52.0,Agir ensemble,Agir ensemble,AGIR-E,...,agnesfirminlebodo,NaN,1.0,4 ans,0.25,0.53,0.918,0.723,2021-08-22,f
2,PA642868,15.0,M.,Christophe,Paul,1971-02-10,50.0,Agir ensemble,Agir ensemble,AGIR-E,...,Paul-Christophe-Député-du-Nord-1387587231318843,paulchristophe.fr/,1.0,4 ans,0.27,0.71,0.944,0.649,2021-08-22,m
3,PA642935,15.0,M.,Becht,Olivier,1976-04-28,45.0,Agir ensemble,Agir ensemble,AGIR-E,...,OlivierBecht68,www.becht-groff2017.fr,1.0,4 ans,0.12,0.68,0.966,0.703,2021-08-22,m
4,PA702052,15.0,Mme,Sage,Maina,1975-05-10,46.0,Agir ensemble,Agir ensemble,AGIR-E,...,MainaSageDeputee,www.mainasage.fr,2.0,7 ans,0.14,0.26,0.977,0.545,2021-08-22,f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568,PA721094,15.0,M.,Dunoyer,Philippe,1968-01-11,53.0,UDI et Indépendants,UDI et Indépendants,UDI_I,...,philippedunoyer2017,NaN,1.0,4 ans,0.08,0.17,0.741,0.500,2021-08-22,m
569,PA721110,15.0,Mme,Sanquer,Nicole,1972-06-16,49.0,UDI et Indépendants,UDI et Indépendants,UDI_I,...,nicolesanquerfareata,NaN,1.0,4 ans,0.08,0.14,0.769,0.545,2021-08-22,f
570,PA721486,15.0,M.,Naegelen,Christophe,1983-12-30,37.0,UDI et Indépendants,UDI et Indépendants,UDI_I,...,ChristopheNaegelenVosges,www.naegelen.fr,1.0,4 ans,0.13,0.14,0.925,0.386,2021-08-22,m
571,PA783463,15.0,Mme,Métadier,Sophie,1961-04-26,60.0,UDI et Indépendants,UDI et Indépendants,UDI_I,...,NaN,NaN,1.0,5 mois,0.05,NaN,NaN,0.571,2021-08-22,f


In [3]:
df1.rename(columns={"nom": "last_name", "prenom": "first_name", "groupeAbrevSansNI":"group", "groupeSansNI":"group_name", "departementCode":"postal_code", "naissance":'birthday', "mail":"email", "experienceDepute":"deputy_years", "datePriseFonction":"deputy_since", "id":"source_id"}, inplace=True)

In [4]:
df1.sample(1).T

,489
source_id,PA722390
legislature,15.0
civ,Mme
last_name,Corneloup
first_name,Josiane
birthday,1959-09-30 00:00:00
age,61.0
groupe,Les Républicains
group_name,Les Républicains
groupeAbrev,LR


In [5]:
df1.shape

(573, 28)

In [6]:
df1.twitter.isna().sum()


62

In [7]:
df1.dropna(subset=['twitter'],inplace=True)
df1.shape

(511, 28)

In [8]:
df1.twitter.value_counts()

antoineherth       1
damienadam76       1
flprovendier       1
JMaireofficiel     1
BeaudouinSo        1
                  ..
jacqlinedubois     1
dmartindijon       1
MonicaMichel_Of    1
FCBDeputeduCher    1
sonjoachim         1
Name: twitter, Length: 511, dtype: int64

In [9]:
client = MongoClient(conf['mongo_uri'])
db = client.politweet

def get_account(account_dict):
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper('from:{}'.format(account_dict['twitter'])).get_items()):
        account = json.loads(tweet.user.json())
        field_to_del = ['_type', 'linkTcourl']
        field_to_add = ['source_id','last_name','first_name','birthday','group_name','group','postal_code','deputy_since','deputy_years','job','email','facebook','website','sex']
        for field in field_to_del: 
            account.pop(field, None)
        for field in field_to_add:
            account[field] = account_dict[field]
        return account
    

def get_oldest_date_tweet(to_str=False):
    res = list(db.tweets.find({},{'date':1}).sort('date', 1).limit(1))
    date = res[0]['date']
    if to_str:
        date = date.strftime('%A %d/%m/%Y à %H:%M') # lundi 04/05/2020 à 12h51
    return date    

def build_tweet(tweet, account, party=False):
    if party:
        tweet_dict = {
            'group':account['group'],
            'account_username':account['username']
        }
    else:        
        tweet_dict = {
            'group':account['group'],
            'account_username':account['username'],
            'account_first_name':account['first_name'],
            'account_last_name':account['last_name'],
            'account_id':account['_id'],
            # 'content_en':translate(tweet.content)
        }
    fields_to_keep = ['url','date','content','renderedContent','id','replyCount','retweetCount','likeCount','quoteCount','conversationId','lang','sourceLabel','hashtags','cashtags']
    fields_to_bool = ['retweetedTweet','quotedTweet','inReplyToTweetId','mentionedUsers']
    for field in fields_to_keep:
        if hasattr(tweet, field):
            tweet_dict[field] = getattr(tweet, field)
    for field in fields_to_bool:
        if hasattr(tweet, field):
            tweet_dict[field] = bool(getattr(tweet, field))
    return tweet_dict

def scrape_tweets(account, since, until, max_tweets=None):
    print("start scraping since {} until {}".format(since, until))
    tweets = []
    params = {'from': account['username']}
    for k,v in {'since_time':since, 'until_time':until}.items():
        params[k] = int(datetime.datetime.timestamp(v))
    match = [':'.join([key,str(val)]) for key,val in params.items()]
    match = ' '.join(match)
    print(account['displayname'])
    print(match)
    # for i,tweet in enumerate(sntwitter.TwitterSearchScraper(match).get_items()):
    #     if max_tweets is not None and i > max_tweets:
    #         break
    #     tweets.append(build_tweet(tweet, account))
    # if len(tweets) > 0:
    #     db.tweets.insert_many(tweets)
    # else:
    #     print("no tweets") 



def scraping_parties(df):
    db.party_tweets.drop()
    parties_dict = df.to_dict('records')
    # print(parties_dict)
    session = db.scrap_sessions.find_one()
    since = session['since']
    until = session['until']
    for party_dict in parties_dict:
        account = {
            'group': party_dict['group'],
            'username': party_dict['twitter'],
        }
        tweets = []
        params = {'from': party_dict['twitter']}
        for k,v in {'since_time':since, 'until_time':until}.items():
            params[k] = int(datetime.datetime.timestamp(v))
        match = [':'.join([key,str(val)]) for key,val in params.items()]
        match = ' '.join(match)
        print(match)
        for i,tweet in enumerate(sntwitter.TwitterSearchScraper(match).get_items()):        
            tweets.append(build_tweet(tweet, account, party=True))
        if len(tweets) > 0:
            db.party_tweets.insert_many(tweets)
        else:
            print("no tweets")


def start_scraping(accounts_df, reset_db=False, scrap_accounts=True, continue_with=None):
    # reset db
    # if reset_db:
    #     db.accounts.drop()
    #     db.tweets.drop()
    #     db.scrap_sessions.drop()
    # log scrapping
    if continue_with:
        session = db.scrap_sessions.find_one()
    else:
        session = {}
        session['until'] = datetime.datetime.now()
        session['since'] = session['until'] - datetime.timedelta(days=365*2)
        db.scrap_sessions.insert_one(session)
    # scrap accounts
    if scrap_accounts:
        accounts_dict = accounts_df.to_dict('records')
        accounts = []
        for account_dict in accounts_dict:
            account = get_account(account_dict)
            if account:
                accounts.append(account)
            else:
                print("scraping error on this account: https://twitter.com/{}".format(account_dict['twitter']))
        db.accounts.insert_many(accounts)
    # scrap tweets
    accounts = db.accounts.find()
    for account in accounts:
        if continue_with and account['username'] == continue_with:
            continue_with = None
        if not continue_with:
            scrape_tweets(account, since=session['since'], until=session['until'])    

In [16]:
scraping_parties(df_party)

[{'group': 'AGIR-E', 'twitter': 'AgirEnsemble_AN'}, {'group': 'DEM', 'twitter': 'MoDem'}, {'group': 'DLF', 'twitter': 'DLF_Officiel'}, {'group': 'EDS', 'twitter': 'edsassnat'}, {'group': 'FI', 'twitter': 'FranceInsoumise'}, {'group': 'GDR', 'twitter': 'deputesPCF'}, {'group': 'GE', 'twitter': 'GEcologie'}, {'group': 'LAREM', 'twitter': 'enmarchefr'}, {'group': 'LDS', 'twitter': 'Liguedusud'}, {'group': 'LND', 'twitter': 'NDemocrates_'}, {'group': 'LR', 'twitter': 'lesRepublicains'}, {'group': 'LT', 'twitter': 'GroupeLibTerrAN'}, {'group': 'RN', 'twitter': 'RNational_off'}, {'group': 'SOC', 'twitter': 'socialistesAN'}, {'group': 'UDI_I', 'twitter': 'UDI_off'}]
from:AgirEnsemble_AN since_time:1566620921 until_time:1629692921
from:MoDem since_time:1566620921 until_time:1629692921
from:DLF_Officiel since_time:1566620921 until_time:1629692921
from:edsassnat since_time:1566620921 until_time:1629692921
from:FranceInsoumise since_time:1566620921 until_time:1629692921
from:deputesPCF since_time

In [45]:
# import logging
# logger = logging.getLogger()
# logger.disabled = True
start_scraping(df1, scrap_accounts=False)
print("Scraping done!")

start scraping since 2019-08-24 06:28:41.731000 until 2021-08-23 06:28:41.731000
Alain Tourret
no tweets
start scraping since 2019-08-24 06:28:41.731000 until 2021-08-23 06:28:41.731000
Monique Iborra
start scraping since 2019-08-24 06:28:41.731000 until 2021-08-23 06:28:41.731000
François de Rugy
start scraping since 2019-08-24 06:28:41.731000 until 2021-08-23 06:28:41.731000
Jacqueline Maquet
start scraping since 2019-08-24 06:28:41.731000 until 2021-08-23 06:28:41.731000
Gwendal ROUILLARD
start scraping since 2019-08-24 06:28:41.731000 until 2021-08-23 06:28:41.731000
Thierry SOLERE
start scraping since 2019-08-24 06:28:41.731000 until 2021-08-23 06:28:41.731000
Jacques Krabal
start scraping since 2019-08-24 06:28:41.731000 until 2021-08-23 06:28:41.731000
Christophe Castaner
start scraping since 2019-08-24 06:28:41.731000 until 2021-08-23 06:28:41.731000
Éric Alauzet
start scraping since 2019-08-24 06:28:41.731000 until 2021-08-23 06:28:41.731000
Richard Ferrand
start scraping sinc

OperationFailure: you are over your space quota, using 517 MB of 512 MB, full error: {'ok': 0, 'errmsg': 'you are over your space quota, using 517 MB of 512 MB', 'code': 8000, 'codeName': 'AtlasError'}

In [39]:
accounts = db.accounts.find()
for account in accounts:
    print(account['username'])

antoineherth
agnesfirmin
christophe_p
becht_olivier
MainaSage
VincentLedoux59
ALouisDeputee13
LoicKervran18
lauredlr
JCLarsonneur
chapelierannie
ChristopheEuzet
PhilippeHuppe
ainakuric
lise_magnier
dhoubron1759
BenoitPotterie
mjidelguerrab
pybournazel
tgassilloud
michelfanget
VigierPhilippe
fpupponi
laqhila
NadiaEssayan
chrisjerretie
BJoncour
PBertaGard
JeanLucLAGLEIZE
erwanbalanant
SophieMette1
SabineThillaye
elodiejala
fabien_laine
GeismarLuc
_richardramos
phbolo
GarciaLaxou
Bannier_G
BHammouche5708
EnMarche_56
MDeprezAudebert
LVichnievsky
josypoueyto
jp_mattei
SylvainWaserman
bru_vincent
Justine_BENIN
MaxMathiasin
fpetitAN
MaudPetit_AN94
BBrocard
f_personne
PatrickMignola
PhilippeMichelK
iflorennes92
maudgatel
MicheleCrouzet
jnbarrot
MdeVaucouleurs
BrunoMillienne
platombe
PLoiseau85
TurquoisNicolas
bruno_fuchs
CIsaacSibille
dupontaignan
emenard34
M_Orphelin
PaulaForteza
AlbaneGaillot
VillaniCedric
MartineWonner
JLMelenchon
Clem_Autain
BenedictTaurine
Michel_Larive
PrudhommeLoic
MRes